# Задание 1: реализуйте задачу классификации на основе BERT-like модели и KNN на данных Russian Intents Dataset с Kaggle.

Цель: научиться создавать классификаторы текстов в условиях большого числа маленьких классов, состоящих из коротких текстов.

Результат: код для создания поискового векторного индекса + логика определения класса на основе близости к обучающим объектам (по ближайшему, по топ-N ближайших, и т. п.).

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "kortana"
os.environ['KAGGLE_KEY'] = "a343264181f7e41c1b4ac43285fc2840"
!kaggle datasets download -d constantinwerner/qa-intents-dataset-university-domain

qa-intents-dataset-university-domain.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
archive = 'qa-intents-dataset-university-domain.zip'
with zipfile.ZipFile(archive, 'r') as zip_file:
    zip_file.extractall('')

In [ ]:
!pip install transformers
!pip install nmslib

# Токенизация тренировочного набора и обучение nmslib

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
import pandas as pd

df_train = pd.read_csv('dataset_train.tsv', sep='\t', header = None)
df_train.head()

,0,1
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general


In [ ]:
#сократим df_train, поскольку полностью датасет не помещается в память при использовании модели
df_train = df_train.sample(frac=1).reset_index().loc[:2000]

In [ ]:
sentences = list(df_train[0].values)
sentences[:5]

['дискаунт на обучение где есть',
 'число стипуха где тут находится',
 'потерять пропуск цифровой',
 'кружок зарегистрировать',
 'отыскать стол военноучетный']

In [ ]:
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)

sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
sentence_embeddings[:5]

tensor([[-0.2294, -0.1451, -0.0106,  ..., -0.1986,  0.4570, -0.0705],
        [-0.1418, -0.1971, -0.0607,  ..., -0.2919,  0.5465, -0.1305],
        [-0.1655, -0.2846,  0.0102,  ..., -0.1636,  0.5550, -0.3928],
        [-0.2055, -0.3350,  0.1381,  ..., -0.3587,  0.5996, -0.2480],
        [-0.2373, -0.0703,  0.0233,  ..., -0.1684,  0.4735, -0.1379]])

In [ ]:
import nmslib

index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(sentence_embeddings, ids=list(range(len(sentence_embeddings))))
index.createIndex({'post': 2}, print_progress=True)

In [ ]:
sentences[0]

'дискаунт на обучение где есть'

In [ ]:
ids, distances = index.knnQuery(sentence_embeddings[0], k=5)
for i, d in zip(ids, distances):
    print(sentences[i], '\t', d, i)

дискаунт на обучение где есть 	 0.0 0
скидка на обучение где есть 	 0.013079405 1186
дискаунт на обучение взять 	 0.01468575 408
нужен дискаунт на обучение 	 0.015127182 401
расположен где дискаунт на обучение 	 0.016408384 1487


# Классификация тестового набора с использованием 5 ближайших соседей

In [ ]:
df_test = pd.read_csv('dataset_test.tsv', sep='\t', header = None)
df_test.head()

,0,1
0,как получить справку,statement_general
1,мне нужна справка,statement_general
2,справка студента эф петь,conform
3,справка студента фф оформлять,conform
4,как мне заказать справка об обучении,conform


In [ ]:
sentences_test = list(df_test[0].values)
sentences_test[:5]

['как получить справку',
 'мне нужна справка',
 'справка студента эф петь',
 'справка студента фф оформлять',
 'как мне заказать справка об обучении']

In [ ]:
encoded_input = tokenizer(sentences_test, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)

sentence_embeddings_test = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
#функция для поиска наиболее часто встречаемого значения
def most_frequent(List):
    return max(set(List), key = List.count)

In [ ]:
classification_test = []
for sent in sentence_embeddings_test:
  ids, distances = index.knnQuery(sent, k=5)
  ids_label = [df_train[1].values[i] for i in ids]
  most_freq = most_frequent(ids_label)
  classification_test.append(most_freq)

In [ ]:
df_test['prediction'] = classification_test
df_test.head()

,0,1,prediction
0,как получить справку,statement_general,conform
1,мне нужна справка,statement_general,conform
2,справка студента эф петь,conform,conform
3,справка студента фф оформлять,conform,conform
4,как мне заказать справка об обучении,conform,conform


In [ ]:
print('Точность:', sum(df_test[1]==df_test.prediction)/len(df_test))

Точность: 0.6455266138165345


С помощью поиска ближайщих соседей удалось квалифицировать тестовый набор данных с хорошей точностью.